In [23]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import unicodedata
import os
import webbrowser
import html5lib
from openpyxl import workbook
from datetime import datetime
import requests

In [28]:
path = '../Master_tables_MFIC_Investment.xlsx'
# dataframes = pd.read_excel(path, sheet_name=None)

xls = pd.ExcelFile(path)
all_sheets = pd.read_excel(path, keep_default_na=False, na_values=[
                           '_'], sheet_name=None)

dataframes = {}
# Loop through each sheet and create a DataFrame in the dictionary
for sheet_name, sheet_df in all_sheets.items():
    dataframes[sheet_name.replace(' ', '_')] = sheet_df
    print(f"DataFrame name: {sheet_name} : {sheet_df.shape}")


process_tables = {}
process_tables_shape = {}
if not os.path.exists('../PT_csv_file'):
    os.makedirs('../PT_csv_file')


def run_process_function(dataframes, process_tables, process_tables_shape):
    path = '../process_tables_GBDC_Investment.xlsx'
    writer = pd.ExcelWriter(path=path, engine='openpyxl')
    for dataframe in dataframes:
        print(dataframe)
        processed_table = process_table_function(dataframes[dataframe])
        process_tables[dataframe] = processed_table
        process_tables_shape[dataframe] = processed_table.shape
        processed_table.to_excel(
            writer, sheet_name=dataframe.replace(',', ''), index=False)
        processed_table.to_csv(
            '../PT_csv_file/'+dataframe.replace(',', '')+'.csv')
        writer.book.save(path)
    writer.close()


def shape(count, df):
    print(f"{count} : shape : {df.shape}")
    count += 1
    return count


def dropna_col_row(df):
    df = df.dropna(how='all', axis=0).reset_index(drop=True)
    df = df.dropna(how='all', axis=1).reset_index(drop=True)
    return df


def drop_if_contain(pattern, df):
    matching_rows = df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    df = df[~matching_rows]
    return df


def rename_columns(df):
    num_cols = df.shape[1]
    data_col_mapper = dict(
        zip(df.columns.to_list(), [i for i in range(0, num_cols)]))
    df = df.rename(columns=data_col_mapper)
    return df

DataFrame name: September 30 2023 : (983, 45)
DataFrame name: June 30 2023 : (1043, 45)
DataFrame name: March 31 2023 : (986, 45)
DataFrame name: December 31 2022 : (890, 45)
DataFrame name: September 30 2022 : (950, 45)
DataFrame name: June 30 2022 : (961, 50)
DataFrame name: March 31 2022 : (915, 50)
DataFrame name: December 31 2021 : (935, 50)
DataFrame name: September 30 2021 : (982, 50)
DataFrame name: June 30 2021 : (1050, 50)
DataFrame name: March 31 2021 : (1006, 50)
DataFrame name: December 31 2020 : (1023, 50)
DataFrame name: September 30 2020 : (1059, 50)
DataFrame name: June 30 2020 : (1109, 50)
DataFrame name: March 31 2020 : (1114, 50)
DataFrame name: December 31 2019 : (872, 27)
DataFrame name: September 30 2019 : (639, 21)
DataFrame name: June 30 2019 : (582, 21)
DataFrame name: March 31 2019 : (492, 21)
DataFrame name: December 31 2018 : (422, 21)
DataFrame name: September 30 2018 : (403, 26)
DataFrame name: June 30 2018 : (369, 21)
DataFrame name: March 31 2018 : (367

In [80]:
def process_table_function(soi_table_df):
    count = 1
    count = shape(count, soi_table_df)
    soi_table_df = soi_table_df.replace(
        r'^\s*\$\s*$', '', regex=True).replace(r'\n', '', regex=True)
    soi_table_df = soi_table_df.replace('-', '0')
    soi_table_df = dropna_col_row(soi_table_df)
    soi_table_df = soi_table_df.apply(
        lambda x: x.strip() if isinstance(x, str) else x)
    count = shape(count, soi_table_df)

    # drops all the extra top row
    pattern = r'Industry'
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Check if the pattern exists in the DataFrame
    if matching_rows.any():
        # Extract rows from the first occurrence onwards
        soi_table_df = soi_table_df.iloc[matching_rows.idxmax():].reset_index(
            drop=True)
    count = shape(count, soi_table_df)

    # drops all the extra bottom row
    pattern = r'Total\s+Investments'
    # Use the apply function to check if the pattern is in any column for each row
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Find the index of the first row that matches the pattern
    # Slice the DataFrame to keep only the rows up to and including the first matching row
    if soi_table_df[matching_rows].index[0] < 20:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[1]].reset_index(
            drop=True)
    else:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[0]].reset_index(
            drop=True)
    count = shape(count, soi_table_df)

    # drop nan col row
    soi_table_df = dropna_col_row(soi_table_df)
    count = shape(count, soi_table_df)

    # drops the sub total
    soi_table_df = soi_table_df.dropna(subset=[soi_table_df.columns[0]])
    count = shape(count, soi_table_df)

    soi_table_df = soi_table_df.replace('', np.nan)
    col_indices = [0]
    soi_table_df.iloc[:, col_indices] = soi_table_df.iloc[:, col_indices].fillna(
        method='ffill')
    count = shape(count, soi_table_df)

    for index, row in soi_table_df.iterrows():
        cleanedList = [x for x in list(row) if str(x) != 'nan']
        row = pd.Series(cleanedList)
        soi_table_df.loc[index] = row

    # Separate header and data
    soi_table_df = soi_table_df.rename(
        columns=soi_table_df.iloc[0]).drop(soi_table_df.index[0])
    # drops all the rows that contains header
    soi_table_df = soi_table_df[soi_table_df[soi_table_df.columns[0]]
                                != soi_table_df.columns[0]]

    soi_table_df = dropna_col_row(soi_table_df)
    count = shape(count, soi_table_df)

    pattern = r'(?:Spread\s*Above|cost|Percentage|Above)'
    soi_table_df = drop_if_contain(pattern, soi_table_df)
    pattern = r'^([Tt]otal)'
    soi_table_df = drop_if_contain(pattern, soi_table_df)
    count = shape(count, soi_table_df)

    soi_table_df.insert(0, 'new_Industry', '')

    for index, row in soi_table_df.iterrows():
        if row.nunique() == 2:
            soi_table_df.at[index, 'new_Industry'] = row.iloc[1]

    # soi_table_df['new_Industry'] = soi_table_df['new_Industry'].replace('', np.nan)

    return soi_table_df


run_process_function(dataframes=dataframes, process_tables=process_tables,
                     process_tables_shape=process_tables_shape)

September_30_2023
1 : shape : (983, 45)
2 : shape : (983, 45)
3 : shape : (952, 45)
4 : shape : (633, 45)
5 : shape : (633, 45)
6 : shape : (633, 45)
7 : shape : (633, 45)
8 : shape : (615, 9)
9 : shape : (589, 9)
Advertising, Printing & Publishing
FingerPaint Marketing
Hero Digital
Automotive
Club Car Wash
Crowne Automotive
K&N Parent, Inc.
Truck-Lite Co., LLC
Aviation and Consumer Transport
Merx Aviation Finance, LLC
Primeflight
Beverage, Food & Tobacco
Berner Foods
Bolthouse Farms
Hive
Orgain, Inc.
Rise Baking
Turkey Hill
Business Services
Accelerate Learning
Access Information
AlpineX
Ambrosia Buyer Corp.
AML Rightsource
Continuum
Electro Rent Corporation
Escalent
Go1
HMA
IRP
Jacent
Jones & Frank
Naviga
PSE
PSI Services, LLC
SEER
Soliant
Trench Plate
US Legal Support
Wilson Language Training
Chemicals, Plastics & Rubber
Carbonfree Chemicals SPE I LLC (f/k/a Maxus Capital Carbon SPE I)
Westfall Technik, Inc.
Construction & Building
Allstar Holdings
Englert
Pave America
Yak Access
Co

/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_23804/639468079.py:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)


2 : shape : (1043, 45)
3 : shape : (1011, 45)
4 : shape : (668, 45)
5 : shape : (668, 45)
6 : shape : (668, 45)
7 : shape : (668, 45)
8 : shape : (649, 9)
9 : shape : (624, 9)
Advertising, Printing & Publishing
FingerPaint Marketing
Hero Digital
Automotive
Club Car Wash
Crowne Automotive
K&N Parent, Inc.
Truck-Lite Co., LLC
Aviation and Consumer Transport
Merx Aviation Finance, LLC
Primeflight
Beverage, Food & Tobacco
Berner Foods
Bolthouse Farms
Hive
Orgain, Inc.
Rise Baking
Turkey Hill
Business Services
Accelerate Learning
Access Information
AlpineX
Ambrosia Buyer Corp.
AML Rightsource
Continuum
Electro Rent Corporation
Elo Touch
Escalent
Go1
HMA
IRP
Jacent
Jones & Frank
Naviga
PSE
PSI Services, LLC
SEER
Soliant
Trench Plate
US Legal Support
Wilson Language Training
Chemicals, Plastics & Rubber
Carbonfree Chemicals SPE I LLC (f/k/a Maxus Capital Carbon
Westfall Technik, Inc.
Construction & Building
Allstar Holdings
Englert
Pave America
Yak Access
Consumer Goods - Durable
A&V
KDC
KLO 

/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_23804/639468079.py:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)


3 : shape : (954, 45)
4 : shape : (618, 45)
5 : shape : (618, 45)
6 : shape : (618, 45)
7 : shape : (618, 45)
8 : shape : (601, 9)
9 : shape : (575, 9)
Advertising, Printing & Publishing
FingerPaint Marketing
Hero Digital
Automotive
Club Car Wash
Crowne Automotive
K&N Parent, Inc.
Truck-Lite Co., LLC
Aviation and Consumer Transport
Merx Aviation Finance, LLC
PrimeFlight
Beverage, Food & Tobacco
Berner Foods
Bolthouse Farms
Hive
Orgain, Inc.
Rise Baking
Turkey Hill
Business Services
Accelerate Learning
Access Information
AlpineX
Ambrosia Buyer Corp.
AML Rightsource
Continuum
Electro Rent Corporation
Elo Touch
Ensemble Health
HMA
IRP
Jacent
Jones & Frank
Naviga
PSE
PSI Services, LLC
Soliant
Trench Plate
US Legal Support
Wilson Language Training
Chemicals, Plastics & Rubber
Carbonfree Chemicals SPE I LLC (f/k/a Maxus Capital Carbon SPE I LLC)
Westfall Technik, Inc.
Construction & Building
Englert
Pave America
Yak Access
Consumer Goods - Durable
A&V
KDC
KLO Holdings, LLC
NSi Industries
Sor

...

/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_23804/639468079.py:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)


March_31_2013
1 : shape : (185, 15)
2 : shape : (185, 15)
3 : shape : (185, 15)
4 : shape : (140, 15)
5 : shape : (140, 15)
6 : shape : (140, 15)
7 : shape : (140, 15)
8 : shape : (135, 6)
9 : shape : (122, 6)
CORPORATE DEBT 0 138.7
SECURED DEBT 0 68.5
1st Lien Secured Debt 0 21.9
2nd Lien Secured Debt 0 46.6
UNSECURED DEBT 0 70.2
UNSECURED DEBT 0 (continued)
STRUCTURED PRODUCTS 0 0.6
PREFERRED EQUITY 0 0.7
EQUITY 0 3.9
Common Equity/Interests 0 3.4
Common Equity/Interests 0 (continued)
Warrants 0 0.5


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_23804/639468079.py:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)


June_30_2013
1 : shape : (229, 23)
2 : shape : (229, 23)
3 : shape : (229, 23)
4 : shape : (151, 23)
5 : shape : (151, 23)
6 : shape : (151, 23)
7 : shape : (151, 23)
8 : shape : (149, 6)
9 : shape : (135, 6)
CORPORATE DEBT0133.8
SECURED DEBT071.0
1st Lien Secured Debt028.8
2nd Lien Secured Debt042.2
2nd Lien Secured Debt0(continued)
UNSECURED DEBT062.8
UNSECURED DEBT0(continued)
STRUCTURED PRODUCTS02.2
PREFERRED EQUITY00.7
EQUITY03.6
Common Equity/Interests03.0
Warrants00.6


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_23804/639468079.py:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)


In [ ]:
df = process_tables['September_30_2023']
numeric_mask = pd.to_numeric(df['Maturity Date'], errors='coerce').notna()

df = df[~numeric_mask]
df.header()